# Code Agents

При создании многоагентных систем, у нас есть компромисс между гибкостью и управляемостью системы. Управляемые системы обычно действуют на основе некоторых прописанных переходов и коммуникаций между агентами, в то время как в гибких системах LLM-агенты могут принимать решения и строить последовательности вызова инструкментов на основе своих знаний и опыта.

Такие системы могут быть построены на основе кодогенерации, когда LLM генерирует код на Python для решения задачи, использующий предоставленные инструменты, который затем выполняется в некоторой контролируемой песочнице.

Для демонстрации такого подхода используем фреймворк [smolagents](https://github.com/huggingface/smolagents).

In [12]:
%pip install smolagents[mcp]


   -------------------- ------------------- 1/2 [mcpadapt]
   ---------------------------------------- 2/2 [mcpadapt]

Note: you may need to restart the kernel to use updated packages.


## Простейший пример

Рассмотрим простейший пример, когда мы даём агенту возможность поиска в интернет. Для этого будем использовать доступный "из коробки" инструмент `DuckDuckGoSearchTool`.

Для работы с YandexGPT будем использовать [OpenAI-совместимое API](https://yandex.cloud/ru/docs/foundation-models/concepts/openai-compatibility). Вы можете попробовать модели YandexGPT 5 или Llama.

> **ВАЖНО**: При использовании YandexGPT 5 необходимо передать ключ `flatten_messages_as_text = True`, поскольку составные запросы, включающие текст и изображения, пока не поддерживаются.

Для начала убедимся, что языковая модель работает:

In [ ]:
from smolagents import OpenAIServerModel, CodeAgent, DuckDuckGoSearchTool
import os

ygpt_model = OpenAIServerModel(api_key=os.environ['api_key'], 
                           model_id=f"gpt://{os.environ['folder_id']}/yandexgpt/rc",
                           #model_id=f"gpt://{os.environ['folder_id']}/llama/rc",
                           api_base="https://llm.api.cloud.yandex.net/v1",
                           flatten_messages_as_text = True
)

ygpt_model([
    {
        "role" : "user",
        "content" : "Расскажи анекдот"
    }])

c:\winapp\conda\envs\mas\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Два друга разговаривают:\n\n— Ты знаешь, как мне удалось так похудеть?\n\n— Нет, как?\n\n— Я стал есть, когда голоден, и перестал есть, когда насытился.\n\n— Так просто?\n\n— Да, но самое сложное было начать делать это каждый день.', tool_calls=None, raw=ChatCompletion(id='f502a002-b0f1-4d1c-9433-445b947691cb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Два друга разговаривают:\n\n— Ты знаешь, как мне удалось так похудеть?\n\n— Нет, как?\n\n— Я стал есть, когда голоден, и перестал есть, когда насытился.\n\n— Так просто?\n\n— Да, но самое сложное было начать делать это каждый день.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1746341834, model='gpt://b1gbicod0scglhd49qs0/yandexgpt/rc', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=64, pro

Теперь создадим агента с инструментом поиска в интернет, и зададим ему какой-нибудь вопрос:

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=ygpt_model)

agent.run("Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены                                       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = web_search(query="best wines for steak with blood and their prices")                                   
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The Ultimate Guide to Steak and Wine 
Pairing](https://goldensteer.com/blogs/journal/the-ultimate-guide-to-steak-and-wine-pairing)
The most elegant cut of steak pairs best with wines that are smooth and refined—not too bold, not too shy. Perfect 
Wine Pairings: Pinot Noir - Light, earthy, ... Sale price $289.00. Add To Cart The Sammy 16 OZ New Yorks (Set of 4)
4.4. Sale price $282.00. Add To Cart

[The Best Wines to Drink with Steak - GAYOT](https://www.gayot.com/wines/top-best-wines-for-pairing-with-steak/)
Origin: Napa Valley, California Varietals: 32% Cabernet Sauvignon, 33% Cabernet Franc, 29% Merlot, 6% Petit Verdot 
Price: $90.00 ABV: 14.1% The Paramount Red Blend is a wine equally driven by Cabernet Franc and Merlot as it is by 
Cabernet Sauvignon. Each component is carefully selected as the best-of-the-best from Gamble's estate vineyards and
is harvested, fermented and aged separately ...

[Which wine for steak? - The Environmental Literacy Council](https://enviroliteracy.org/which-wine-for-steak/)
Which Wine for Steak? A Carnivore's Guide to Perfect Pairings. Choosing the right wine to complement a delicious 
steak is a culinary art form, balancing flavors and textures to elevate your dining experience. The best wine for 
steak isn't a one-size-fits-all answer; it depends heavily on the cut of meat, its preparation, and any 
accompanying sauces or seasonings.

[The Vivino guide to wine pairing with steak: how to find the perfect 
match](https://www.vivino.com/wine-news/the-vivino-guide-to-wine-pairing-with-steak-how-to-find-the-perfect-match)
A rich, fatty cut of steak will go beautifully with a bold wine that has tannins, while a leaner steak might 
benefit from a lighter, more delicate wine. Let's take a look at some of the best wine with steak pairings to take 
your steak dinner to the next level. Classic red wine and steak combinations. Let's start with the red wine 
classics ...

[A Starter Guide to Wine and Steak 
Pairing](https://www.wineenthusiast.com/basics/how-to-pair/wine-and-steak-pairing/)
Best Wine for a Ribeye Steak. Ribeye is the richest cut of steak, lusciously marbled with fat throughout. This is 
where you can really play with the interaction between tannins and beef fat. Price recommends Napa's hillside AVAs,
like Diamond Mountain. "These wines are so rugged and burly, they almost seem chewy, with a wet-dark earthiness," 
she says.

[Steak and Wine: The Best Red Wine to Pair With Beef - Robb 
Report](https://robbreport.com/food-drink/wine/gallery/best-steak-wine-1236321179/)
Subscribe now and get up to 61% off the cover price. ... The 7 Best Wines to Pair With Your Next Steak ... they 
purchased the Goosecross to set their roots more permanently. They make about 20 ...

[The Definitive Guide For Pairing Wine And Steak, According To A Master 
...](https://www.tastingtable.com/1774983/guide-pairing-wine-steak-master-sommelier/)
"Generally, red wine pairs best with steak," acknowledges Ballance. "However, if you are partial to white wine , 
you can select a fuller bodied white when pairing your steak dishes."

[Best wine with steak: What to choose - 
Decanter](https://www.decanter.com/learn/advice/wine-steak-ask-decanter-400770/)
For meat lovers, an evening with a succulent, juicy steak and a delicious bottle of wine is one of life's great 
pleasures. The big meaty flavours of a steak demand an equally big wine with plenty of tannins - and the good news 
is there are several red grapes and wine styles that fit the bill.. As with any food pairing, it's also useful to 
consider the other ingredients on your plate.

[Uncork the Perfect Match: Top Wines for Steak Lovers | Pairing 
Guide](https://redwinedecanter.com/2024/09/14/best-wine-with-steak/)
Discover the perfect wine to elevate your steak dinner experience. From bold Cabernets to smooth Malbecs, explore 
expert pairings for different cuts and cooking methods. Learn serving tips and find budget-friendly to 
splurg

[Step 1: Duration 3.57 seconds| Input tokens: 2,320 | Output tokens: 72]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  wines.append({"name": "Cabernet Sauvignon", "price": "$100.00"})                                                 
  final_answer(wines)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: [{'name': 'Pinot Noir', 'price': '$289.00'}, {'name': 'The Paramount Red Blend', 'price': 
'$90.00'}, {'name': 'Cabernet Sauvignon', 'price': '$100.00'}]

[Step 4: Duration 2.80 seconds| Input tokens: 15,584 | Output tokens: 373]

[{'name': 'Pinot Noir', 'price': '$289.00'},
 {'name': 'The Paramount Red Blend', 'price': '$90.00'},
 {'name': 'Cabernet Sauvignon', 'price': '$100.00'}]

Несмотря на то, что мы задали вопрос на русском языке, весь поиск в интернет проводился на английском, и был найден результат из англоязычного сегмента. Почему?

Дело в том, что за кажущейся внешней простотой скрывается большая внутренняя работа. Во фреймворк smolagents заложена [идеология ReAct](https://habr.com/ru/articles/882432/), в которой агент сначала рассуждает и строит план решения, затем по шагам выполняет его, смотрит на результаты и адаптируется. За этим стоит достаточно сложный промптинг, который по умолчанию во фреймворке smolagents сделан на английском языке.

Загрузим русскоязычный пакет промптов:

In [ ]:
import yaml
russian_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)
agent.prompt_templates = russian_templates
agent.run("Напиши список из трёх лучших вин, подходящих к стейку с кровью, и их цены")

Видим, что результат получился уже на русском языке.

## Model Context Protocol

В примере выше, мы искали цены где-то в интернет. А как быть, если агенту нужно работать с конкретным рестораном?

Для этого можно использовать технологию [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction). Это по сути дела стандартизированный способ сделать "удалённый RAG" и "удалённый вызов инструментов". В этом случае ресторан может предоставлять своё меню через набор MCP-вызовов. Для этого он должен реализовать MCP-сервер.

В нашем примере предположим, что у нас есть ресторан, с некоторым [меню кухни](data/menu/food.md) и [бара](data/menu/drinks.md). Мы реализовали [простой MCP-сервер](mcp-server/mcp-rest.py), который предоставляет два инструмента:
* `get_food_menu` для получения списка блюд в виде markdown-таблицы
* `get_drinks_menu` - для получения списка напитков

Для запуска сервера необходимо установить [FastMCP](https://github.com/jlowin/fastmcp), и далее в командной строке выполнить:
```bash
fastmcp run -p 8000 -h 127.0.0.1 -t sse mcp-rest.py
```

In [62]:
from smolagents import OpenAIServerModel, CodeAgent, DuckDuckGoSearchTool
import os
import yaml

russian_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)

ygpt_model = OpenAIServerModel(api_key=os.environ['api_key'], 
                           model_id=f"gpt://{os.environ['folder_id']}/yandexgpt/rc",
                           #model_id=f"gpt://{os.environ['folder_id']}/llama/rc",
                           api_base="https://llm.api.cloud.yandex.net/v1",
                           flatten_messages_as_text = True)

Теперь мы можем использовать MCP-инструменты в нашем агенте:

In [11]:
import yaml
from smolagents.mcp_client import MCPClient

with MCPClient({"url" : "http://127.0.0.1:8000/sse"}) as tools:
    agent = CodeAgent(tools=tools, model=ygpt_model)
    agent.prompt_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)
    agent.run("Какое самое дорогое блюдо в ресторане?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Какое самое дорогое блюдо в ресторане?                                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  menu = get_food_menu()                                                                                           
  print(menu)                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
| Название блюда            | Описание                                                                 | Тип  | 
Цена (руб) |  
|-------------------------------|-----------------------------------------------------------------------------|----
------|---------------|  
| "Бык на взводе"           | Сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей. Подаётся с 
золотой солью Гималаев. | Стейк    | 2500          |  
| "Нежность разъярённой коровы" | Филе-миньон, замаринованное в гранатовом соке и шёпоте шеф-повара. Плюс картофель
"А-ля крылышки дракона". | Стейк    | 1800          |  
| "Стейк для скромного олигарха" | Говяжья щека, томлёная 12 часов в вине из погреба Наполеона (ну почти). С 
хрустящими чипсами из кабачков. | Стейк    | 1200          |  
| "Лосось в мечтах о Норвегии" | Нежнейший филе-кусок, запечённый под корочкой из "загадочных северных трав" 
(укропа). Подаётся с лимонным бризом. | Рыба     | 1900          |  
| "Осьминог, который видел слишком много" | Гриль-кальмар с чернильным соусом и намёком на средиземноморские тайны.
Для смелых гурманов. | Рыба     | 2200          |  
| "Дотерь под шубой"        | Филе дорадо, завёрнутое в хрустящий картофельный панцирь. С соусом "Тайна Атлантиды".
| Рыба     | 1600          |  
| "Картофель 'Я тоже могу быть трюфелем'" | Нежные дольки, запечённые с розмарином и намёком на дороговизну. 
Подаётся с воздухом Пиреней. | Гарнир   | 600           |  
| "Грибы, которые танцевали на луну" | Шампиньоны-гриль с чесночным соусом и намёком на психоделический опыт. | 
Гарнир   | 450           |  
| "Круассаны, но не те"     | Аппетитные мини-морковки, глазированные мёдом с намёком на парижское детство. | 
Гарнир   | 350           |  
| "Шоколадный апокалипсис"  | Десерт из 72% какао, взрывающий вкусовые рецепторы. С мороженым "Облако рая". | 
Десерт   | 800           |  
| "Чизкейк, который видел Бога" | Воздушный, с карамелью из ангельских слез и крошками "райского печенья". | Десерт
| 750           |  
| "Тирамису для взрослых"   | Классика, но с добавлением "секретного ингредиента" (кофе, а что вы подумали?). | 
Десерт   | 700           |  


Out: None

[Step 1: Duration 2.17 seconds| Input tokens: 2,281 | Output tokens: 63]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
                                                                                                                   
  # Извлечем цены из строки меню                                                                                   
  prices = re.findall(r'\|\s*(\d+)\s*\|', menu)                                                                    
  prices = [int(price) for price in prices]                                                                        
                                                                                                                   
  # Найдем максимальную цену                                                                                       
  max_price = max(prices)                                                                                          
  print(f"Максимальная цена: {max_price} руб.")                                                                    
                                                                                                                   
  # Найдем блюдо с максимальной ценой                                                                              
  most_expensive_dish = re.search(rf'\|\s*(.+?)\s*\|\s*{max_price}\s*\|', menu).group(1)                           
  final_answer(f"Самое дорогое блюдо: {most_expensive_dish} ({max_price} руб.)")                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Максимальная цена: 2500 руб.

Out - Final answer: Самое дорогое блюдо: | "Бык на взводе"           | Сочный рибай с розовым сердцем, томлёный в 
дыме аргентинских страстей. Подаётся с золотой солью Гималаев. | Стейк (2500 руб.)

[Step 2: Duration 3.82 seconds| Input tokens: 5,344 | Output tokens: 267]

## Подбор меню для ужина

Теперь попробуем решить более сложную задачу - подобрать меню для ужина таким образом, чтобы вино и основное блюдо хорошо сочетались. Для этого будем использовать [табличку соответствий еды и вина](data/food_wine_table.md):

In [28]:
import pandas as pd
import re 

foodmatch = [ x.strip() for x in open("data/food_wine_table.md", encoding="utf8").readlines()[2:-1]]
foodmatch = [ { "food" : x.split('|')[0].strip(), "wine" : x.split('|')[1].strip()} for x in foodmatch]
df = pd.DataFrame(foodmatch)
df

,food,wine
0,"Баклажаны, запеченые с сыром",Красное вино: «среднетелые»* сухие — Гренаш (Г...
1,Баранина деликатесная (филе или каре ягненка),Красное вино: сухие выдержанные вина из виногр...
2,"Баранина пикантная: жареная, гриль, тушеная — ...",Красные вина: сухие вина из винограда Каберне ...
3,Бефстроганов,"Белые вина: выдержанные в дубе Шардоне, Пино Г..."
4,Блинчики с мясом говядина,"Крепкие напитки: Водка, Полугар, Хреновуха. Т..."
...,...,...
155,"Шашлык из свинины, пикантный — в уксусно-луков...",Красные вина: тихие сухие и полусухие вина из ...
156,Шоколад,"Чай: черный, травяной, ягодно-фруктовый. Не об..."
157,Эклер классический,"Белые вина: более сладкое, чем торт выдержанно..."
158,Яичница — глазунья,Белые вина: сухие легкие вина с невысокой кисл...


Чтобы научить нашего агента искать соответствия еды и вина, нам нужно дать ему соответствующие инструменты. А для этого реализовать какой-то вид поиска поверх нашей таблички.

Будем использовать семантический поиск через YandexGPT Embeddings:

In [26]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(folder_id=os.environ['folder_id'], auth=os.environ['api_key'])
query_model = sdk.models.text_embeddings('query')
doc_model = sdk.models.text_embeddings('doc')

Посчитаем эмбеддинги для всех блюд и вин в табличке:

In [ ]:
df['food_embed'] = df['food'].apply(lambda x : doc_model.run(x))
df['wine_embed'] = df['wine'].apply(lambda x : doc_model.run(x))
df.to_pickle("data/food_wine_table.pkl")

In [36]:
df = pd.read_pickle("data/food_wine_table.pkl")

Теперь собственно опишем функцию поиска подходящей еды к вину `find_matching_food`:

In [59]:
from scipy.spatial.distance import cdist
import numpy as np
from smolagents import tool

@tool
def find_matching_food(wine : str) -> str:
    """
    Найти подходщее блюдо к заданному вину
    Args:
        wine: Описание вина, под которое надо найти блюдо
    """
    wine_embed = query_model.run(wine)
    we = np.array([np.array(x) for x in df['wine_embed']])
    dist = cdist([wine_embed], we, metric='cosine')[0]
    return "Вот наиболее подходящие блюда:\n" + "\n".join(
        [f"{j+1}. {df.iloc[i]['food']}" 
        for j, i in enumerate(dist.argsort()[0:3])])

print(find_matching_food("Сухое белое"))


Вот наиболее подходящие блюда:
1. Нарезка мясная деликатесная варено-копченая (в т.ч. колбаса)
2. Сыры козие молодые
3. Рыба белая речная (судак, щука, жерех, толстолобик, карп)


Аналогичная функция для поиска вина:

In [60]:
@tool
def find_matching_wine(food : str) -> str:
    """
    Найти подходщее вино к заданному блюду
    Args:
        food: Описание блюда, под которое надо найти вино
    """
    food_embed = query_model.run(food)
    fe = np.array([np.array(x) for x in df['food_embed']])
    dist = cdist([food_embed], fe, metric='cosine')[0]
    return "Вот наиболее подходящие вина:\n" + "\n".join(
        [f"{j+1}. {df.iloc[i]['wine']}" 
        for j, i in enumerate(dist.argsort()[0:3])])

print(find_matching_wine('Стейк'))

Вот наиболее подходящие вина:
1. Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше.
2. Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).
3. Красные вина: сухие вина из винограда Санджовезе (Кьянти) и Неро д’Авола, испанские Темпранильо, «тельные» Мерло, «тельные» Пино Нуары (из Нового Света, Австрии, Герма

Теперь собираем всё это вместе, и создаём агента, которому доступны как инструменты подбора блюд/вин, так и меню ресторана по протоколу MCP.

In [67]:
with MCPClient({"url" : "http://127.0.0.1:8000/sse"}) as res_tools:
    agent = CodeAgent(
        tools=res_tools+[find_matching_food,find_matching_wine], 
        prompt_templates = russian_templates,
        model=ygpt_model)
    agent.prompt_templates = yaml.load(open('etc/smolagents_code_agent_russian_prompts.yaml'), Loader=yaml.FullLoader)
    agent.run("""
    Подбери меню для ужина в ресторане из основного блюда, подходящего вина и десерта, и
    посчитай его стоимость.
    Выведи табличку из названий блюж и напитков и их стоимости. Я хочу стейк!
    """)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Подбери меню для ужина в ресторане из основного блюда, подходящего вина и десерта, и                            │
│     посчитай его стоимость.                                                                                     │
│     Выведи табличку из названий блюж и напитков и их стоимости. Я хочу стейк!                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  steak_dish = "стейк"                                                                                             
  matching_wine = find_matching_wine(steak_dish)                                                                   
  print(f"Подходящее вино к {steak_dish}: {matching_wine}")                                                        
  food_menu = get_food_menu()                                                                                      
  print("Меню блюд ресторана:")                                                                                    
  print(food_menu)                                                                                                 
  drinks_menu = get_drinks_menu()                                                                                  
  print("Меню напитков ресторана:")                                                                                
  print(drinks_menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Подходящее вино к стейк: Вот наиболее подходящие вина:
1. Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от
Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые 
аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне 
Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный 
«ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше.
2. Красные вина: сухие и полусухие вина из винограда Гарнача (Гренаш), Мерло, Карменер, Менсия, «тельные» Пино 
Нуары (со всего света), российский Красностоп, Гамэ (Божоле Виляж).
3. Красные вина: сухие вина из винограда Санджовезе (Кьянти) и Неро д’Авола, испанские Темпранильо, «тельные» 
Мерло, «тельные» Пино Нуары (из Нового Света, Австрии, Германии), вина из Бордо, вина из Роны 
(Гренаш+Сира+Мурведр), Саперави (как из Грузии, так и из России), Арени (Армения).
Меню блюд ресторана:
| Название блюда            | Описание                                                                 | Тип  | 
Цена (руб) |  
|-------------------------------|-----------------------------------------------------------------------------|----
------|---------------|  
| "Бык на взводе"           | Сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей. Подаётся с 
золотой солью Гималаев. | Стейк    | 2500          |  
| "Нежность разъярённой коровы" | Филе-миньон, замаринованное в гранатовом соке и шёпоте шеф-повара. Плюс картофель
"А-ля крылышки дракона". | Стейк    | 1800          |  
| "Стейк для скромного олигарха" | Говяжья щека, томлёная 12 часов в вине из погреба Наполеона (ну почти). С 
хрустящими чипсами из кабачков. | Стейк    | 1200          |  
| "Лосось в мечтах о Норвегии" | Нежнейший филе-кусок, запечённый под корочкой из "загадочных северных трав" 
(укропа). Подаётся с лимонным бризом. | Рыба     | 1900          |  
| "Осьминог, который видел слишком много" | Гриль-кальмар с чернильным соусом и намёком на средиземноморские тайны.
Для смелых гурманов. | Рыба     | 2200          |  
| "Дотерь под шубой"        | Филе дорадо, завёрнутое в хрустящий картофельный панцирь. С соусом "Тайна Атлантиды".
| Рыба     | 1600          |  
| "Картофель 'Я тоже могу быть трюфелем'" | Нежные дольки, запечённые с розмарином и намёком на дороговизну. 
Подаётся с воздухом Пиреней. | Гарнир   | 600           |  
| "Грибы, которые танцевали на луну" | Шампиньоны-гриль с чесночным соусом и намёком на психоделический опыт. | 
Гарнир   | 450           |  
| "Круассаны, но не те"     | Аппетитные мини-морковки, глазированные мёдом с намёком на парижское детство. | 
Гарнир   | 350           |  
| "Шоколадный апокалипсис"  | Десерт из 72% какао, взрывающий вкусовые рецепторы. С мороженым "Облако рая". | 
Десерт   | 800           |  
| "Чизкейк, который видел Бога" | Воздушный, с карамелью из ангельских слез и крошками "райского печенья". | Десерт
| 750           |  
| "Тирамису для взрослых"   | Классика, но с добавлением "секретного ингредиента" (кофе, а что вы подумали?). | 
Десерт   | 700           |  

Меню напитков ресторана:
| Сорт вина / Напиток       | Винодельня / Производитель       | Год | Тип           | Цена за бокал (руб) |  
|------------------------------|--------------------------------------|---------|--------------------|-------------
-----------|  
| Каберне Совиньон         | Château Lafite Rothschild, Франция   | 2018    | Красное вино       | 4500            
|  
| Мерло                    | Marchesi Antinori, Италия            | 2019    | Красное вино       | 2800            
|  
| Пино Нуар                | Domaine de la Romanée-Conti, Франция | 2017    | Красное вино       | 5200            
|  
| Шираз                    | Penfolds, Австралия                  | 2018    | Красное вино       | 3200            
|  
| Маль

[Step 1: Duration 3.27 seconds| Input tokens: 2,419 | Output tokens: 145]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  selected_steak = "Бык на взводе"                                                                                 
  selected_wine = "Каберне Совиньон"                                                                               
  selected_dessert = "Шоколадный апокалипсис"                                                                      
                                                                                                                   
  steak_price = 2500                                                                                               
  wine_price = 4500                                                                                                
  dessert_price = 800                                                                                              
                                                                                                                   
  total_cost = steak_price + wine_price + dessert_price                                                            
  print(f"Общая стоимость ужина: {total_cost} руб.")                                                               
                                                                                                                   
  final_answer(f"| Блюдо / Напиток           | Цена (руб) |\n|-------------------------------|---------------|\n|  
  {selected_steak} | {steak_price} |\n| {selected_wine} | {wine_price} |\n| {selected_dessert} | {dessert_price}   
  |")                                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Общая стоимость ужина: 7800 руб.

Out - Final answer: | Блюдо / Напиток           | Цена (руб) |
|-------------------------------|---------------|
| Бык на взводе | 2500 |
| Каберне Совиньон | 4500 |
| Шоколадный апокалипсис | 800 |

[Step 2: Duration 4.22 seconds| Input tokens: 6,782 | Output tokens: 371]

## Заключение

Code Agents - это способ создания достаточно гибких агентов, способных самостоятельно декомпозировать задачу на подзадачи и решать их с помощью написания кода. 